In [ ]:
# Importing libraries
import pickle
import os
import re
import gc
import datetime
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from google.cloud import bigquery
from matplotlib import pyplot as plt
import google.oauth2.credentials
from google.cloud import bigquery
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
local_dev = True

In [ ]:
project_id = 'bi-stg-mobilityds-pr-db8ce2'
proj_id = 'divg-team-v03-pr-de558a'
dataset_id = 'ym'

In [ ]:
def get_gcp_bqclient(project_id, use_local_credential=False):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client
def plotImp(model, X,figsize, num):
    feature_imp = pd.DataFrame({'Value':model.feature_importance(),'Feature':X.columns})
    plt.figure(figsize=figsize)
    sns.set(font_scale = 1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [ ]:
client = get_gcp_bqclient(proj_id, use_local_credential=True)

In [ ]:
# Initializing constants
# client = bigquery.Client(project=proj_id)

# CUST_ID, Bus_Billing_Account_Num, Bus_Prod_Instnc_Id, ADTbT_, ADT_Gazelle_Clients, ADT_FLAG, Base_Snapshot_Date, Security_Origin, Cultural_Segment, Installation_Type, Product_Profile, Province_Grp, Tenure_Month_Groups, ACQUIRED_FROM, DIY_Activated, price_plan, Acquisition_Source, Activation_Ts__Month___Year_, bi_chnl_tag_cd, Channel, Channel__Best_, Channel_Category, CHNL_ORG_TXT__Custom_SQL_Query_, Commitment_Type, Community, Contract_Expiring_Soon, Contract_Type, Cont

trainingSQL = f"""
 select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_v3`
     where Base_Snapshot_Date between '2023-01-01' and '2023-09-30' and security_origin='Organic'
"""    #    ='2023-10-01'  order by rand() limit 500000
valSQL = f"""
 select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_v3`
     where  Base_Snapshot_Date between '2023-10-01'and '2023-10-31' and security_origin='Organic'
""" 

trainingSQL = f"""
 select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_v3`
     where Base_Snapshot_Date between '2023-01-01' and '2023-11-30'
     
"""    #    ='2023-10-01'  order by rand() limit 500000
valSQL = f"""
 select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_v3`
     where Base_Snapshot_Date between '2023-12-01'and '2023-12-31'
""" 

testSQL = f"""
 select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_v3`
     where  Base_Snapshot_Date between '2024-01-01' and '2024-01-31'
""" 

# test_sql_1 = f"""
#     select distinct * except(invol, churn,Month_name,Sales_Agent_ID,src_usr_chnl_txt, yr_mth_1,
#      yr_mth_2,Activation_Ts__Month___Year_,
#      yr_mth_3) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final` 
#      where  Base_Snapshot_Date = '2023-11-01'
# """

# test_sql_2 = f"""
#     select distinct * except(invol, churn,Month_name,Sales_Agent_ID,src_usr_chnl_txt, yr_mth_1,
#      yr_mth_2,Activation_Ts__Month___Year_,
#      yr_mth_3) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final` 
#      where  Base_Snapshot_Date = '2023-12-01'
# """ 

#order by rand() limit 200000 
# left join labels by ban + sub level

In [ ]:
# def data_prep(main_df,scoring_start_date):
#     df=main_df.copy()
#     for col in df.columns:
#         df[col] = list(df[col])
        
#     df = df.T.drop_duplicates().T
#     bool_cols = ['ADTbT_','ADT_Gazelle_Clients','ADT_FLAG','Contracted__Account_','HasFibreTag', 'HasHP', 'HasHSIA',
#                'HasIPTV', 'HasKoodo', 'HasMobility', 'HasPik', 'HasPublic',
#                'HasSMHM', 'HasT_Post', 'HasTOS', 'HasTV', 'HasWHP', 'HasWHSIA',
#                'HP_CNT', 'HS_CNT','TOS_FLAG','Video_','Data_flag','Best_practices'
#             ]
#     date_cols = ['Base_Snapshot_Date']
#     cat_cols = ['Security_Origin','Cultural_Segment','Installation_Type',
#                 'Product_Profile','Province_Grp','Tenure_Month_Groups','Price_Plan_Txt__SHS_',
#                'ACQUIRED_FROM','DIY_Activated','price_plan','Acquisition_Source',
#                 'Activation_Ts__Month___Year_','bi_chnl_tag_cd','Channel',
#                 'Channel__Best_','Channel_Category','CHNL_ORG_TXT__Custom_SQL_Query_',
#                 'Commitment_Type','Community','Contract_Expiring_Soon','Contract_Type',
#                 'CREDIT_VALUE_CD__group_','Customer_Category','Delivery_Technology',
#                 'Delivery_Technology__Account_','POSTAL_CD__Custom_SQL_Query_','Security_Plan_Group',
#                 'Tenure_Month_Groups__All_','Tenure_Year_Group','TV_Count','community_nm',
#                 'census_subdivision_typ','census_subdivision_desc','prizm_name','social_grp_cd','Security_Plan_Type',
#                 'social_grp_nm','lifestage_cd','lifestage_nm','prov_cd','Segment','order_date','munic_nm','chnl_org_txt',
#                 'prov_state_cd','Sales_Channel_tag','txn_sub_typ_txt','SLS_ACTVY_TXT','PRICE_PLAN_TXT__Custom_SQL_Query_'
               
#                ]
#     num_cols = ['Epp_Discount_Months','HSLTE_TOTAL', 'IPTV_CNT','KBRAND_BAN_CT','Nho_Discount_Months',
#                'avg_income', 'med_income','Login_consistency','Arming_consistency'
               
#                ]
#     num_cols += [col for col in df.columns if '_amt_' in col.lower()]
#     target = ['vol']
    
#     le_dict = {}
#     for col in bool_cols+cat_cols:
#         # print(col)
#         df[col] = df[col].fillna('unknown').astype(str)
#         le = LabelEncoder().fit(df[col])
#         le_dict[col] = le
#         df[col] = le.transform(df[col])
#     for col in num_cols:
#         df[col] = df[col].fillna(df[col].median()).astype(float)

#     other_cols = [col for col in df.columns if col not in bool_cols+date_cols+cat_cols+num_cols+target]
#     df.drop(columns=other_cols,axis=1, inplace=True)

#     uni_cols=[]
#     for col in df.columns:
#         if df[col].nunique()==1:
#             uni_cols.append(col)
#     df.drop(columns=uni_cols,axis=1, inplace=True)
#     return df

In [ ]:
# Pulling data from BigQuery
train_df = client.query(trainingSQL).to_dataframe()
val_df = client.query(valSQL).to_dataframe()
# val_df = val_df.T.drop_duplicates().T
test_df = client.query(testSQL).to_dataframe()
# test_df = test_df.T.drop_duplicates().T

# evaluate_df = client.query(eval_sql).to_dataframe() 
# testing_df = client.query(test_sql).to_dataframe() 
# print(train_df.shape,val_df.shape,test_df.shape,evaluate_df.shape,testing_df.shape)
# print(train_df.shape,val_df.shape,test_df.shape)

print(train_df.shape,val_df.shape,test_df.shape)

In [ ]:
train_df.columns.values

In [ ]:
# n_train, n_val,n_test = train_df.shape[0], val_df.shape[0], test_df.shape[0] #,evaluate_df.shape[0],testing_df.shape[0]
# df = pd.concat([train_df,val_df,test_df,evaluate_df,testing_df],axis=0)
# df = pd.concat([train_df,val_df,test_df],axis=0)

n_train, n_val, n_test = train_df.shape[0], val_df.shape[0], test_df.shape[0]
df = pd.concat([train_df,val_df,test_df],axis=0)

In [ ]:
del train_df,val_df,test_df  #,,test_df evaluate_df,testing_df
gc.collect()

In [ ]:
df.shape

In [ ]:
for col in df.columns:
    df[col] = list(df[col])

In [ ]:
bool_cols = ['ADTbT_','ADT_Gazelle_Clients','ADT_FLAG','Contracted__Account_','HasFibreTag', 'HasHP', 'HasHSIA',
               'HasIPTV', 'HasKoodo', 'HasMobility', 'HasPik', 'HasPublic',
               'HasSMHM', 'HasT_Post', 'HasTOS', 'HasTV', 'HasWHP', 'HasWHSIA',
               'HP_CNT', 'HS_CNT','TOS_FLAG','Video_','Best_practices'    # 'Data_flag',
            ]
date_cols = ['Base_Snapshot_Date']
cat_cols = ['Security_Origin','Cultural_Segment','Installation_Type',
            'Product_Profile','Province_Grp','Tenure_Month_Groups','Price_Plan_Txt__SHS_',
           'ACQUIRED_FROM','DIY_Activated','Acquisition_Source',   #,'price_plan'
            'bi_chnl_tag_cd','Channel',  #'Month_name',
            'Channel__Best_','Channel_Category','CHNL_ORG_TXT__Custom_SQL_Query_',
            'Commitment_Type','Community','Contract_Expiring_Soon','Contract_Type',
            'CREDIT_VALUE_CD__group_','Customer_Category','Delivery_Technology',
            'Delivery_Technology__Account_','Security_Plan_Group',   #'POSTAL_CD__Custom_SQL_Query_',
            'Tenure_Month_Groups__All_','Tenure_Year_Group','TV_Count','community_nm',
            'census_subdivision_typ','census_subdivision_desc','prizm_name','social_grp_cd','Security_Plan_Type',
            'social_grp_nm','lifestage_cd','lifestage_nm','prov_cd','munic_nm','chnl_org_txt','Segment',
            'prov_state_cd','Sales_Channel_tag','txn_sub_typ_txt','SLS_ACTVY_TXT','PRICE_PLAN_TXT__Custom_SQL_Query_'
           
           ]
num_cols = ['Epp_Discount_Months','HSLTE_TOTAL', 'IPTV_CNT','KBRAND_BAN_CT','avg_income', 
            'med_income',  'Login_consistency', 'Arming_consistency','days_since_order_date',
            'call_cnt_r30d',
           'avg_talk_time_r30d', 'avg_hold_time_r30d', 'avg_emp_cnt_r30d',
           'avg_esc_cnt_r30d', 'std_talk_time_r30d', 'std_hold_time_r30d',
           'std_emp_cnt_r30d', 'std_esc_cnt_r30d', 'sum_talk_time_r30d',
           'sum_hold_time_r30d', 'sum_emp_cnt_r30d', 'sum_esc_cnt_r30d',
           'max_talk_time_r30d', 'max_hold_time_r30d', 'max_emp_cnt_r30d',
           'call_cnt_r30d_smhm', 'avg_talk_time_r30d_smhm',
           'avg_hold_time_r30d_smhm', 'avg_emp_cnt_r30d_smhm',
           'avg_esc_cnt_r30d_smhm', 'std_talk_time_r30d_smhm',
           'std_hold_time_r30d_smhm', 'std_emp_cnt_r30d_smhm',
           'std_esc_cnt_r30d_smhm', 'sum_talk_time_r30d_smhm',
           'sum_hold_time_r30d_smhm', 'sum_emp_cnt_r30d_smhm',
           'sum_esc_cnt_r30d_smhm', 'max_talk_time_r30d_smhm',
           'max_hold_time_r30d_smhm', 'max_emp_cnt_r30d_smhm',
           'max_esc_cnt_r30d_smhm', 'hs_dl_usg_gb_l12m', 'hs_ul_usg_gb_l12m',
           'hs_tot_usg_gb_l12m', 'hs_dl_usg_gb_1', 'hs_ul_usg_gb_1',
           'hs_tot_usg_gb_1', 'hs_dl_usg_gb_2', 'hs_ul_usg_gb_2',
           'hs_tot_usg_gb_2', 'hs_dl_usg_gb_3', 'hs_ul_usg_gb_3',
           'hs_tot_usg_gb_3', 'tckt_cnt', 'dispatch_cnt', 'sev1_cnt',
           'sev2_cnt', 'sev3_cnt', 'sev4_cnt', 'sing_cnt', 'hsic_cnt',
           'ttv_cnt', 'streamplus_cnt', 'whsia_cnt', 'smhm_cnt', 'tos_cnt',
           'livingwell_cnt', 'install_cnt', 'web_portal_cnt', 'billing_cnt',
           'qual_cnt', 'facilities_cnt', 'other_cnt', 'days_close_min',
           'days_close_max', 'days_close_avg', 'days_resolve_min',
           'days_resolve_max', 'days_resolve_avg', 'tot_click_cnt_r30d',
           'wls_tot_cnt_r30d', 'wls_plan_cnt_r30d', 'wls_device_cnt_r30d',
           'wln_tot_cnt_r30d', 'wln_eligibility_cnt_r30d',
           'wln_sing_cnt_r30d', 'wln_hsic_cnt_r30d', 'wln_fibre_cnt_r30d',
           'wln_whsia_cnt_r30d', 'wln_wifi_plus_cnt_r30d', 'wln_tv_cnt_r30d',
           'wln_optik_cnt_r30d', 'wln_pik_cnt_r30d',
           'wln_streamplus_cnt_r30d', 'wln_streaming_cnt_r30d',
           'wln_security_cnt_r30d', 'wln_smarthome_security_cnt_r30d',
           'wln_online_security_cnt_r30d', 'wln_smartwear_security_cnt_r30d',
           'deal_tot_cnt_r30d', 'deal_wls_cnt_r30d', 'deal_wln_cnt_r30d',
           'deal_wln_security_cnt_r30d', 'upgr_tot_cnt_r30d',
           'upgr_wls_cnt_r30d', 'upgr_wln_cnt_r30d',
           'upgr_wln_security_cnt_r30d', 'chg_tot_cnt_r30d',
           'chg_wls_cnt_r30d', 'chg_wln_cnt_r30d',
           'chg_wln_security_cnt_r30d',
               
           ]
num_cols += [col for col in df.columns if '_amt_' in col.lower()]
# target = ['vol_ctr_ind']
target = ['vol']

 # exclude below
 # data_flag
 # Login_consistency
 # Arming_consistency
 # Best_practices
 # Month_name
 # Segment

In [ ]:
le_dict = {}
for col in bool_cols+cat_cols:
    # print(col)
    df[col] = df[col].fillna('unknown').astype(str)
    le = LabelEncoder().fit(df[col])
    le_dict[col] = le
    df[col] = le.transform(df[col])
    
for col in num_cols:
    df[col] = df[col].fillna(df[col].median()).astype(float)
    
other_cols = [col for col in df.columns if col not in bool_cols+cat_cols+num_cols+target and col not in ['call_to_retention_ind','vol','vol_ctr_ind']]
df.drop(columns=other_cols,axis=1, inplace=True)
uni_cols=[]
for col in df.columns:
    if col in ['call_to_retention_ind','vol','vol_ctr_ind']: continue
    if df[col].nunique()==1:
        uni_cols.append(col)
df.drop(columns=uni_cols,axis=1, inplace=True)    

In [ ]:
df[target].value_counts()/len(df)

In [ ]:
features = [col for col in df.columns if col not in ['vol','Base_Snapshot_Date','order_date','POSTAL_CD__Custom_SQL_Query_','call_to_retention_ind','vol_ctr_ind']]
len(features)

In [ ]:
df_train = df[:n_train]
df_val = df[n_train:(n_train+n_val)]
df_test = df[(n_train+n_val):(n_train+n_val+n_test)]

df_train.shape, df_val.shape, df_test.shape

In [ ]:
X_train = df_train[features]
y_train = list(np.squeeze(df_train[target].values))

X_val = df_val[features]
y_val = list(np.squeeze(df_val[target].values))

X_test = df_test[features]
y_test = list(np.squeeze(df_test[target].values))

In [ ]:
X_train.shape, len(y_train) ,X_val.shape, len(y_val) ,X_test.shape, len(y_test)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
test_data = lgb.Dataset(X_test, label=y_test)

# del X_train, X_val, df_train, df_test, df_val
gc.collect()
# --- params for Organic
# param = {'n_jobs':4,
#         'learning_rate':0.02,
#         'objective':'binary',
#         'num_leaves':32,
#         'num_threads':4,
#         'max_depth':8,
#         'tree_learner':'serial',
#         'feature_fraction':0.7,
#         'bagging_freq':5,
#         'bagging_fraction':0.7,
#         'verbose':100,
#         "verbosity": -1,
#         'bagging_seed':11,
#         'metric': ['auc', 'binary_logloss'],
#         }

# ------ params for migrated
param = {'n_jobs':4,
        'learning_rate':0.05,
        'objective':'binary',
        'num_leaves':32,
        'num_threads':4,
        'max_depth':4,
        'tree_learner':'serial',
        'feature_fraction':0.7,
        'bagging_freq':5,
        'bagging_fraction':0.7,
        'verbose':100,
        "verbosity": -1,
        'bagging_seed':11,
        'metric': ['auc', 'binary_logloss'],
        }
num_round  = 100000
model_lgb = lgb.train(param,train_data, num_round, valid_sets=[train_data,val_data],callbacks=[lgb.early_stopping(stopping_rounds=200)])


## evaluate model performance



### use test set: 2023-12-01. 

In [ ]:
pred_probs_lgb = model_lgb.predict(X_test,num_iteration=model_lgb.best_iteration)

In [ ]:
def get_lift(prob, y_test, q):
    result = pd.DataFrame(columns=['Prob', 'Churn'])
    result['Prob'] = prob
    result['Churn'] = y_test
    # result['Decile'] = pd.qcut(1-result['Prob'], 10, labels = False)
    result['Decile'] = pd.qcut(result['Prob'], q, labels=[i for i in range(q, 0, -1)])
    add = pd.DataFrame(result.groupby('Decile')['Churn'].mean()).reset_index()
    add.columns = ['Decile', 'avg_real_churn_rate']
    result = result.merge(add, on='Decile', how='left')
    result.sort_values('Decile', ascending=True, inplace=True)
    lg = pd.DataFrame(result.groupby('Decile')['Prob'].mean()).reset_index()
    lg.columns = ['Decile', 'avg_model_pred_churn_rate']
    lg.sort_values('Decile', ascending=False, inplace=True)
    lg['avg_churn_rate_total'] = result['Churn'].mean()
    lg['total_churn'] = result['Churn'].sum()
    lg = lg.merge(add, on='Decile', how='left')
    lg['lift'] = lg['avg_real_churn_rate'] / lg['avg_churn_rate_total']
    
    return lg
    
def calculate_lift(y_true, y_pred, n_bins=10):
    # Create a DataFrame with y_true and y_pred
    data = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})

    # Sort the DataFrame by y_pred in descending order
    data = data.sort_values(by='y_pred', ascending=False)

    # Calculate the base rate
    base_rate = np.mean(y_true)

    # Calculate the lift
    data['cumulative_positive'] = data['y_true'].cumsum()
    data['cumulative_total'] = np.arange(1, len(data) + 1)
    data['cumulative_rate'] = data['cumulative_positive'] / data['cumulative_total']
    data['lift'] = data['cumulative_rate'] / base_rate

    # Calculate lift for each bin
    data['bin'] = pd.qcut(data['cumulative_total'], n_bins, labels=False)
    lift_by_bin = data.groupby('bin')['lift'].mean()

    return lift_by_bin


In [ ]:
lg = get_lift(pred_probs_lgb,y_test,10)
sns.barplot(x='Decile',y='lift',data=lg)
lg

In [ ]:
plotImp(model_lgb, X_train,figsize=(15,8), num=50)

### predict on Nov and Dec clients and evaluate with Jan the same clients

In [ ]:
sql_1 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Nov_2023`
     where  Base_Snapshot_Date = '2023-11-01' and security_origin='Organic'
"""

sql_2 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Dec_2023`
     where  Base_Snapshot_Date = '2023-12-01' and security_origin='Organic'
"""

sql_3 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Jan_2024` 
     where  Base_Snapshot_Date = '2024-01-01' and security_origin='Organic'
"""

In [ ]:
sql_1 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Nov_2023`
     where  Base_Snapshot_Date = '2023-11-01'
"""

sql_2 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Dec_2023`
     where  Base_Snapshot_Date = '2023-12-01'
"""

sql_3 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Jan_2024` 
     where  Base_Snapshot_Date = '2024-01-01'
"""

In [ ]:
client = get_gcp_bqclient(proj_id, use_local_credential=True)

In [ ]:
# df_nov = client.query(sql_1).to_dataframe()
df_dec = client.query(sql_2).to_dataframe()
df_jan= client.query(sql_3).to_dataframe()

In [ ]:
# df_nov = df_nov.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')
df_dec = df_dec.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')
df_jan = df_jan.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')

In [ ]:
# df_nov.shape
df_dec.shape,df_jan.shape

In [ ]:
# df_nov['Bus_Billing_Account_Num'].nunique()==len(df_nov),
df_dec['Bus_Billing_Account_Num'].nunique()==len(df_dec),df_jan['Bus_Billing_Account_Num'].nunique()==len(df_jan)

In [ ]:
for col in df_dec.columns:
    # df_nov[col] = list(df_nov[col])
    df_dec[col] = list(df_dec[col])
    df_jan[col] = list(df_jan[col])

In [ ]:
# # ban_nov = list(df_nov['Bus_Billing_Account_Num'].unique())
# ban_dec = list(df_dec['Bus_Billing_Account_Num'].unique())
# ban_jan = list(df_jan['Bus_Billing_Account_Num'].unique())

# # ban_common = set(ban_nov).intersection(set(ban_dec))
# ban_common = set(ban_dec)
# ban_common = set(ban_common).intersection(set(ban_jan))

# # ban_common = [ban for ban in ban_nov if ban in ban_dec and ban in ban_jan]
# # len(ban_nov),len(ban_dec),len(ban_jan),

# len(ban_common)

In [ ]:
# # df_nov = df_nov.loc[df_nov['Bus_Billing_Account_Num'].isin(list(ban_common))]
# df_dec = df_dec.loc[df_dec['Bus_Billing_Account_Num'].isin(list(ban_common))]
# df_jan = df_jan.loc[df_jan['Bus_Billing_Account_Num'].isin(list(ban_common))]
# # len(df_nov),len(df_dec),len(df_jan)

# # len(df_nov),
# len(df_dec),len(df_jan)

In [ ]:
# df_dec=df_dec.merge(df_jan[['Bus_Billing_Account_Num','vol']],on='Bus_Billing_Account_Num',how='left')

In [ ]:
# df_dec['vol_x'].value_counts()

In [ ]:
# df_dec['vol_y'].value_counts()

In [ ]:
# # y_true_nov = list(np.squeeze(df_nov['vol_x'].values)) #nov label
# # y_true_dec = list(np.squeeze(df_nov['vol_y'].values))  # dec label

# y_true_jan = list(np.squeeze(df_dec['vol_y'].values))  # jan label

# # len(y_true_nov),len(y_true_dec),

# len(y_true_jan)

In [ ]:
y_true_dec = list(np.squeeze(df_dec['vol'].values)) 
y_true_jan = list(np.squeeze(df_jan['vol'].values))

print(len(y_true_dec))
print(len(y_true_jan))

In [ ]:
le_dict

In [ ]:
le_dict_cp = le_dict.copy()

In [ ]:
def preprocessing(df,le_dict_cp):
    df_ = df.copy()
    for col in bool_cols+cat_cols:
        # if col in ['order_date','price_plan','POSTAL_CD__Custom_SQL_Query_','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']: continue
        print(col, end=' ')
        df_[col] = df_[col].fillna('unknown').astype(str)
        # a = set(np.array(le_dict[col].classes_))
        a = set(le_dict_cp[col].classes_)
        # print('before',a)        
        b = set(df_[col].unique())
        new_x = a^b
        a.update(new_x)
        le_dict_cp[col].classes_ = np.array(a)
        df_[col] = le_dict_cp[col].fit_transform(df_[col])
    
    for col in num_cols:
        df_[col] = df_[col].fillna(df_[col].median()).astype(float)
        
    other_cols = [col for col in df_.columns if col not in bool_cols+cat_cols+num_cols+target]
     # and col not in ['call_to_retention_ind','vol','vol_ctr_ind','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']
    other_cols = [col for col in other_cols if col not in features]
    df_.drop(columns=other_cols,axis=1, inplace=True)
    uni_cols=[]
    for col in df_.columns:
        # if col in ['call_to_retention_ind','vol','vol_ctr_ind','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']: continue
        if df_[col].nunique()==1:
            uni_cols.append(col)
    uni_cols = [col for col in uni_cols if col not in features]
    df_.drop(columns=uni_cols,axis=1, inplace=True)  
    return df_

In [ ]:
# df_nov_processed = preprocessing(df_nov,le_dict_cp)

df_dec_processed = preprocessing(df_dec,le_dict_cp)
df_jan_processed = preprocessing(df_jan,le_dict_cp)

In [ ]:
# X_test_nov = df_nov_processed[features]

X_test_dec = df_dec_processed[features]
X_test_jan = df_jan_processed[features]

In [ ]:
# y_pred_nov = model_lgb.predict(X_test_nov,num_iteration=model_lgb.best_iteration)
y_pred_dec = model_lgb.predict(X_test_dec,num_iteration=model_lgb.best_iteration)
y_pred_jan = model_lgb.predict(X_test_jan,num_iteration=model_lgb.best_iteration)

In [ ]:
# lg_1 = get_lift(y_pred_nov,y_true_jan,10)
# sns.barplot(x='Decile',y='lift',data=lg_1)
# lg_1

In [ ]:
lg_2 = get_lift(y_pred_dec,y_true_dec,20)
sns.barplot(x='Decile',y='lift',data=lg_2)
lg_2

In [ ]:
lg_2 = get_lift(y_pred_dec,y_true_dec,10)
sns.barplot(x='Decile',y='lift',data=lg_2)
lg_2

In [ ]:
lg_3 = get_lift(y_pred_jan,y_true_jan,10)
sns.barplot(x='Decile',y='lift',data=lg_3)
lg_3

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
df_ = X_train.copy()
df_['target'] = y_train
corr_df = df_.corr()['target']
corr_df=corr_df.sort_values(ascending=False)
corr_df

In [ ]:
corr_df.head(20)

In [ ]:
feature_imp = pd.DataFrame({'Value':model_lgb.feature_importance(),'Feature':X_train.columns})
feature_imp.sort_values(by="Value", ascending=False,inplace=True)

In [ ]:
importances = model_lgb.feature_importance()

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x = range(len(importances))

feature_names = X_train.columns

# Create tick labels 
labels = np.array(feature_names)[sorted_index]

for idx in sorted_index: 
    print(f'{feature_names[idx]}, {importances[idx]}', end='\n')

In [ ]:
feature_imp.loc[feature_imp.Value>30].head(20)

In [ ]:
feature_imp.loc[feature_imp.Value>60]['Feature'].values